In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')

def GradBShift(signal, x_ele, y_ele, freq):
    
    nch = signal.shape[0]
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    Nsample = signal.shape[1]
    fsample = 200e6
    
    grad_b_angles = 2 * np.pi * np.arange(0, Nsample, 1) * freq / fsample
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)
    
    start_phases = 2 * np.pi * np.sqrt((x_antenna - x_ele) ** 2 + (y_antenna - y_ele) ** 2) / wavelength_lo
    
    x_diff = x_antenna - x_ele
    y_diff = y_antenna - x_ele
    
    theta_ele = np.arctan2(-y_ele, x_ele)
    r_ele = np.sqrt(x_ele ** 2 + y_ele ** 2)
    
    theta_ele_grad_b = theta_ele + grad_b_angles
    #print(theta_ele_grad_b)
    
    x_ele_grad_b = r_ele * np.cos(theta_ele_grad_b)
    y_ele_grad_b = r_ele * np.sin(theta_ele_grad_b)
    
    d_grad_b = np.sqrt((x_antenna.reshape((x_antenna.size, 1)) - x_ele_grad_b) ** 2 + (y_antenna.reshape((y_antenna.size, 1)) - y_ele_grad_b) ** 2)
    
    grad_b_phase = 2 * np.pi * d_grad_b / wavelength_lo
    
    grad_b_only_phases = grad_b_phase - start_phases.reshape((start_phases.size, 1))
    grad_b_shift = np.exp(-1j * grad_b_only_phases)
    
    shifted_signal = grad_b_shift * signal
    
    return shifted_signal
    
    #x_antenna = x_antenna.reshape((x_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    #y_antenna = y_antenna.reshape((y_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    
    #x_grid, y_grid = np.meshgrid(x_range, y_range)
    
    #x_grid = x_grid.flatten()
    #y_grid = y_grid.flatten()
    
    #r_grid = np.sqrt(x_grid ** 2 + y_grid ** 2)
    
    #theta_grid = np.arctan2(-y_grid, x_grid)
    #theta_grid_grad_b = theta_grid.reshape(theta_grid.size, 1) + grad_b_angles
    #theta_grid_grad_b = theta_grid.reshape((x_range.size, y_range.size, 1)).repeat(Nsample, axis=-1) + grad_b_phases
    
    #print(r_grid.shape, theta_grid_grad_b.shape, r_array, angles.shape)
    
    #delta_theta = angles.reshape((angles.size, 1, 1)) - theta_grid_grad_b.reshape((1, *theta_grid_grad_b.shape)) 
    
    #x_grad_b = r_grid.reshape((r_grid.size, 1)) * np.cos(theta_grid_grad_b)
    #y_grad_b = r_grid.reshape((r_grid.size, 1)) * np.sin(theta_grid_grad_b)
    
    #print(x_grad_b.shape, y_grad_b.shape)
    
    #d_grad_b = np.sqrt((x_antenna.reshape((x_antenna.size, 1, 1)) - x_grad_b.reshape((1, *x_grad_b.shape))) ** 2 + (y_antenna.reshape((y_antenna.size, 1, 1)) - y_grad_b.reshape((1, *y_grad_b.shape))) ** 2)
    
    #source_angle = -1 * 2 * np.pi * np.arange(0, Nsample, 1) * freq / fsample
    
    #r_new = np.sqrt(x_new ** 2 + y_new ** 2)
    
    #x_new = r_new * np.cos(source_angle).reshape((1, Nsample)).repeat(nch, axis=0)
    #y_new = r_new * np.sin(source_angle).reshape((1, Nsample)).repeat(nch, axis=0)
    
    #d_old = np.sqrt(x_antenna ** 2 + y_antenna ** 2)
    #d_new = np.sqrt((x_new - x_antenna) ** 2 + (y_new - y_antenna) ** 2)
    
    #phase_shift = 2 * np.pi * (d_grad_b) / wavelength_lo + angles.reshape((angles.size, 1, 1))
    #print(phase_shift.shape, signal.shape)
    
    #shifted_signal = np.exp(-1j * phase_shift) * signal.reshape((signal.shape[0], 1, signal.shape[-1]))
    
    #return shifted_signal.sum(axis=0)
    
    
def ShiftAndSum(signal, x_range, y_range, freq):
    
    nch = signal.shape[0]
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    Nsample = signal.shape[1]
    fsample = 200e6
    
    grad_b_angles = 2 * np.pi * np.arange(0, Nsample, 1) * freq / fsample
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)
    
    #x_antenna = x_antenna.reshape((x_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    #y_antenna = y_antenna.reshape((y_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    
    x_grid, y_grid = np.meshgrid(x_range, y_range)
    
    x_grid = x_grid.flatten()
    y_grid = y_grid.flatten()
    
    r_grid = np.sqrt(x_grid ** 2 + y_grid ** 2)
    
    theta_grid = np.arctan2(-y_grid, x_grid)
    theta_grid_grad_b = theta_grid.reshape(theta_grid.size, 1) + grad_b_angles
    #theta_grid_grad_b = theta_grid.reshape((x_range.size, y_range.size, 1)).repeat(Nsample, axis=-1) + grad_b_phases
    
    #print(r_grid.shape, theta_grid_grad_b.shape, r_array, angles.shape)
    
    #delta_theta = angles.reshape((angles.size, 1, 1)) - theta_grid_grad_b.reshape((1, *theta_grid_grad_b.shape)) 
    
    x_grad_b = r_grid.reshape((r_grid.size, 1)) * np.cos(theta_grid_grad_b)
    y_grad_b = r_grid.reshape((r_grid.size, 1)) * np.sin(theta_grid_grad_b)
    
    #print(x_grad_b.shape, y_grad_b.shape)
    
    d_grad_b = np.sqrt((x_antenna.reshape((x_antenna.size, 1, 1)) - x_grad_b.reshape((1, *x_grad_b.shape))) ** 2 + (y_antenna.reshape((y_antenna.size, 1, 1)) - y_grad_b.reshape((1, *y_grad_b.shape))) ** 2)
    
    #source_angle = -1 * 2 * np.pi * np.arange(0, Nsample, 1) * freq / fsample
    
    #r_new = np.sqrt(x_new ** 2 + y_new ** 2)
    
    #x_new = r_new * np.cos(source_angle).reshape((1, Nsample)).repeat(nch, axis=0)
    #y_new = r_new * np.sin(source_angle).reshape((1, Nsample)).repeat(nch, axis=0)
    
    #d_old = np.sqrt(x_antenna ** 2 + y_antenna ** 2)
    #d_new = np.sqrt((x_new - x_antenna) ** 2 + (y_new - y_antenna) ** 2)
    
    phase_shift = 2 * np.pi * (d_grad_b) / wavelength_lo + angles.reshape((angles.size, 1, 1))
    #print(phase_shift.shape, signal.shape)
    
    shifted_signal = np.exp(-1j * phase_shift) * signal.reshape((signal.shape[0], 1, signal.shape[-1]))
    
    return shifted_signal.sum(axis=0)


shift_signal = GradBShift(np.zeros((60, 8192), np.complex64), 0.02, 0.00, -1e3)


# load data

In [ ]:
os.listdir(os.path.join(DATAPATH))

In [ ]:
os.listdir(os.path.join(RESULTPATH))

In [ ]:
#test_data = mf.data.MFDataset(os.path.join(DATAPATH, '210930_87_18595_energy_angle_test_slice8192.h5'))
template_data = mf.data.MFDataset(os.path.join(DATAPATH, '211027_84_25_2cm.h5'))

#beamforming_result = np.load(os.path.join(RESULTPATH,'211005_mf_84_100_template_ideal_scores_bottom_tri.npy'))

# compute scores

# load meta data

In [ ]:
template_metadata = pd.DataFrame(template_data.metadata)
#test_metadata = pd.DataFrame(test_data.metadata)

In [ ]:
energy_array = template_metadata['energy'].array
angle_array = template_metadata['theta_min'].array

#test_energies = np.array(test_metadata['energy'].array).squeeze()
#test_angles = np.array(test_metadata['theta_min'].array).squeeze()

unique_energy = np.sort(np.unique(template_metadata['energy'].array))
unique_angle = np.sort(np.unique(template_metadata['theta_min'].array))

energy_grid, angle_grid = np.meshgrid(unique_energy, unique_angle)

sorting_indexes = []

for i, pair in enumerate(zip(energy_grid.flatten(), angle_grid.flatten())):
    
    sorting_indexes.append(template_metadata[(template_metadata['energy'] == pair[0]) & (template_metadata['theta_min'] == pair[1])].index[0])
    
sorting_indexes = np.array(sorting_indexes)

print(sorting_indexes.shape)

In [ ]:
print(unique_energy, unique_angle)

# remove grad-b shift, do beamforming, signal 1

In [ ]:
signal_energy = 18595
signal_angle = 89.5 

index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]

signal = template_data.data[index, :].reshape((60, 8192 * 3))


ngrid = 21
x_pos_range = np.linspace(-0.01, -0.03, ngrid)
y_pos_range = np.linspace(-0.01, 0.01, ngrid)
grad_b_freq = 2.5e3

shifted_signal = GradBShift(signal, 0.02, 0.00, -grad_b_freq)
#shifted_signal = GradBShift(signal, 0.02, 0.00, grad_b_freq)

#pos_beam = np.linspace(-xpos_range, pos_range, ngrid)

sum_signal = ShiftAndSum(shifted_signal, x_pos_range, y_pos_range, 0)

In [ ]:
sns.set_theme(style='ticks', context='paper')
cmap = sns.color_palette('mako_r', as_cmap=True)
fig = plt.figure(figsize=(13,13))
ax = fig.add_subplot(1,1,1)

image = np.flip((abs(sum_signal) ** 2).sum(axis=-1).reshape((ngrid, ngrid)), axis=1)
print(image.max())

img = ax.imshow(image, cmap = cmap, aspect='auto', interpolation='none')

cbar = fig.colorbar(img)

# choose signal and template, apply grad-b shift to signal, check matched filter scores

In [ ]:
####

var = 1.38e-23 * 10 * 50 * 200e6
rng = np.random.default_rng()
noise = rng.multivariate_normal([0, 0], np.eye(2) * var / 2, 60 * 8192 * 3 )
noise = noise[:, 0] + 1j * noise[:, 1]

signal_energy = 18595
signal_angle = 89.6

index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]
signal = template_data.data[index, :].reshape((60, 8192 * 3))

template_energy = 18595
template_angle = 89.6

index = template_metadata[(template_metadata['energy'] == template_energy) & (template_metadata['theta_min'] == template_angle)].index[0]
template = template_data.data[index, :].reshape((60, 8192 * 3))

template = (template.flatten() * 1 / np.sqrt(var * np.vdot(template.flatten(), template.flatten()))).reshape((60, 8192 * 3))

nlist = 301
grad_b_freq_list = np.linspace(-2e3, 2e3, nlist)
score_list_1 = np.zeros(nlist)

for i, freq in enumerate(grad_b_freq_list):
    
    noise = rng.multivariate_normal([0, 0], np.eye(2) * var / 2, 60 * 8192 * 3)
    noise = noise[:, 0] + 1j * noise[:, 1]
    
    shift_signal = GradBShift(signal, 0.02, 0.00, -freq)
    
    score = abs(np.vdot(template.flatten(), shift_signal.flatten() + noise ))
    
    score_list_1[i] = score
    
    #print(i + 1)
    
####

template_energy = 18584
template_angle = 89.85

index = template_metadata[(template_metadata['energy'] == template_energy) & (template_metadata['theta_min'] == template_angle)].index[0]
template = template_data.data[index, :].reshape((60, 8192 * 3))

template = (template.flatten() * 1 / np.sqrt(var * np.vdot(template.flatten(), template.flatten()))).reshape((60, 8192 * 3))

score_list_2 = np.zeros(nlist)

for i, freq in enumerate(grad_b_freq_list):
    
    noise = rng.multivariate_normal([0, 0], np.eye(2) * var / 2, 60 * 8192 * 3)
    noise = noise[:, 0] + 1j * noise[:, 1]
    
    shift_signal = GradBShift(signal, 0.02, 0.00, -freq)
    
    score = abs(np.vdot(template.flatten(), shift_signal.flatten() + noise))
    
    score_list_2[i] = score

####

In [ ]:
sns.set_theme(style='whitegrid', context='paper')
clist = sns.color_palette('deep')

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)
ax.plot(grad_b_freq_list, score_list_1 - score_list_1.min(), '.')

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)
ax.plot(grad_b_freq_list, score_list_2 - score_list_2.min(), '.')

#ax.set_xlim(75, 125)
#ax.set_ylim(7, 8)

# fit a polynomial to the different curves to estimate the central position

In [ ]:
def fit_func(x, a, b, c, d):
    
    return d * np.exp(- (x - a) ** 2 / b ** 2) + c

popt_1, pcov_1 = scipy.optimize.curve_fit(fit_func, grad_b_freq_list, score_list_1-score_list_1.min(), [0.0, 1500, 0.0, 10])
popt_2, pcov_2 = scipy.optimize.curve_fit(fit_func, grad_b_freq_list, score_list_2-score_list_2.min(), [0.0, 1500, 0.0, 10])
print(popt_1, popt_2)
print(np.sqrt(pcov_1))
print(np.sqrt(pcov_2))

In [ ]:
sns.set_theme(style='whitegrid', context='paper')
clist = sns.color_palette('deep')

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)
ax.plot(grad_b_freq_list, score_list_1-score_list_1.min(), '.')

ax.plot(grad_b_freq_list, fit_func(grad_b_freq_list, *popt_1))

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)
ax.plot(grad_b_freq_list, score_list_2-score_list_2.min(), '.')

ax.plot(grad_b_freq_list, fit_func(grad_b_freq_list, *popt_2))

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

ax.plot(grad_b_freq_list, fit_func(grad_b_freq_list, *popt_1))
ax.plot(grad_b_freq_list, fit_func(grad_b_freq_list, *popt_2))